Basic example of Aaron's code from [this GitHub Nengo issue discusssion](https://github.com/nengo/nengo/issues/921#issuecomment-374277601).

In [16]:
import time

import numpy as np
from scipy.optimize import nnls

import nengo
from nengo.params import NumberParam
from nengo.solvers import Solver

In [8]:
nengo.rc.set("decoder_cache", "enabled", "False")

In [17]:
class DalesSolver(Solver):
    """Solves for weights subject to Dale's principle."""
    
    # TODO: needs testing (e.g., transforms), support for slicing
    # TODO: currently doesn't support finger-printing by the cache

    p_inh = NumberParam('p_inh', low=0, high=1)
    
    def __init__(self, p_inh=0.2):
        super(DalesSolver, self).__init__(weights=True)
        self.p_inh = p_inh

    def __call__(self, A, Y, rng=None, E=None):
        pre_n_neurons = A.shape[1]
        post_n_neurons = E.shape[1]
        
        tstart = time.time()
        i = int(self.p_inh * pre_n_neurons)

        A[:, :i] *= (-1)
        W = np.empty((pre_n_neurons, post_n_neurons))
        J = Y.dot(E)
        # TODO: make a version that uses nengo.solvers.NnlsL2
        for j in range(post_n_neurons):
            W[:, j], _ = nnls(A, J[:, j])
        W[:i, :] *= (-1)

        return W, {
            'time': time.time() - tstart,
            'i': i,}

In [18]:
tau = 0.1
tau_probe = 0.005
dt = 0.001

with nengo.Network(seed=0) as model:
    stim = nengo.Node(output=lambda t: 2*np.pi*5*np.cos(2*np.pi*5*t))
    
    x = nengo.Ensemble(100, 1)

    # Discrete Principle 3 (Voelker & Eliasmith, 2017; eq 21)
    nengo.Connection(stim, x, transform=dt / (1 - np.exp(-dt/tau)), synapse=tau)
    conn = nengo.Connection(x, x, synapse=tau, solver=DalesSolver())
    
    p_stim = nengo.Probe(stim, synapse=tau_probe)
    p_x = nengo.Probe(x, synapse=tau_probe)

with nengo.Simulator(model, dt=dt, optimize=False) as sim:
    sim.run(1.0)
    
print("Solver Info:", sim.data[conn].solver_info)

Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 
Solver Info: {'time': 0.6614587306976318, 'i': 20}
